In [209]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
import fnmatch
import os
import glob

In [210]:
df = pd.read_csv('data.txt', sep=" ", header=None)
df_normal_marked = pd.read_csv('saved.csv')
df.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]
df = df.dropna()
df.index = df.index + 1

In [211]:
df_normal_marked['severity'] = 'N'

In [212]:
df_final = pd.concat([df, df_normal_marked], axis=0)

In [213]:
df_final

,ID,background_tissue,class,severity,x,y,rad,Unnamed: 0
1,mdb001,G,CIRC,B,535,425,197.0,NaN
2,mdb002,G,CIRC,B,522,280,69.0,NaN
5,mdb005,F,CIRC,B,477,133,30.0,NaN
6,mdb005,F,CIRC,B,500,168,26.0,NaN
11,mdb010,F,CIRC,B,525,425,33.0,NaN
...,...,...,...,...,...,...,...,...
188,mdb318,D,NORM,N,442,68,50.0,202.0
189,mdb319,D,NORM,N,590,281,50.0,203.0
190,mdb320,D,NORM,N,495,264,50.0,204.0
191,mdb321,D,NORM,N,604,271,50.0,205.0


In [214]:
df_final = df_final.reset_index(drop=True)
del df_final["Unnamed: 0"]

In [215]:
df_final

,ID,background_tissue,class,severity,x,y,rad
0,mdb001,G,CIRC,B,535,425,197.0
1,mdb002,G,CIRC,B,522,280,69.0
2,mdb005,F,CIRC,B,477,133,30.0
3,mdb005,F,CIRC,B,500,168,26.0
4,mdb010,F,CIRC,B,525,425,33.0
...,...,...,...,...,...,...,...
307,mdb318,D,NORM,N,442,68,50.0
308,mdb319,D,NORM,N,590,281,50.0
309,mdb320,D,NORM,N,495,264,50.0
310,mdb321,D,NORM,N,604,271,50.0


In [254]:
def img_erode():
    for index, row in df_final.iterrows():
        path = 'all-mias/{name}.pgm'.format(name=row['ID'])
        im = cv2.imread('all-mias/{name}.pgm'.format(name=row['ID']), 0)
        #img = cv2.imread('j.png',0)
        kernel = np.ones((5,5),np.uint8)
        erosion = cv2.erode(im,kernel,iterations=1) 
        cv2.imwrite('erode/{}.png'.format(row['ID']), erosion)
img_erode()

In [231]:
def img_canny():
    for index, row in df_final.iterrows():
        path = 'all-mias/{name}.pgm'.format(name=row['ID'])
        im = cv2.imread('all-mias/{name}.pgm'.format(name=row['ID']))
        img_cvt=cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        fin = cv2.Canny(img_cvt,10,20)
        cv2.imwrite('canny/{}.png'.format(row['ID']), fin)
img_canny()

In [121]:
df_normal = pd.read_csv('data.txt', sep=" ", header=None)
df_normal.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]

In [122]:
df_normal = df_normal.loc[df_normal['class']  == 'NORM']
df_normal = df_normal.reset_index(drop=True)

In [123]:
df_normal

,ID,background_tissue,class,severity,x,y,rad
0,mdb003,D,NORM,NaN,NaN,NaN,NaN
1,mdb004,D,NORM,NaN,NaN,NaN,NaN
2,mdb006,F,NORM,NaN,NaN,NaN,NaN
3,mdb007,G,NORM,NaN,NaN,NaN,NaN
4,mdb008,G,NORM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
202,mdb318,D,NORM,NaN,NaN,NaN,NaN
203,mdb319,D,NORM,NaN,NaN,NaN,NaN
204,mdb320,D,NORM,NaN,NaN,NaN,NaN
205,mdb321,D,NORM,NaN,NaN,NaN,NaN


In [124]:
df['severity'].value_counts()

B    69
M    54
Name: severity, dtype: int64

In [10]:
im = Image.open('all-mias/mdb001.pgm') # PILLOW image processing

In [11]:
im.getpixel( (535,425) )

155

In [110]:
last = 'start'

In [111]:
def img_draw(image, x, y, r, c):
    if c=='M':
        color = 'red'
    else:
        color = 'blue'
    draw = ImageDraw.Draw(image)
    leftUpPoint = (x-r, y-r)
    rightDownPoint = (x+r, y+r)
    twoPointList = [leftUpPoint, rightDownPoint]
    draw.ellipse(twoPointList, outline=color, width=5)

In [ ]:
for index, row in df.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    if last != path:
        im = Image.open(path).convert("RGB")
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    c = row['severity']
    img_draw(im, x, 1024 - y, r, c)
    im.save("marked/{}.png".format(row['ID']))
    last = path

In [147]:
df_final

,ID,background_tissue,class,severity,x,y,rad
0,mdb001,G,CIRC,B,535,425,197.0
1,mdb002,G,CIRC,B,522,280,69.0
2,mdb003,D,NORM,NaN,NaN,NaN,NaN
3,mdb004,D,NORM,NaN,NaN,NaN,NaN
4,mdb005,F,CIRC,B,477,133,30.0
...,...,...,...,...,...,...,...
518,mdb318,D,NORM,NaN,442,68,50.0
519,mdb319,D,NORM,NaN,590,281,50.0
520,mdb320,D,NORM,NaN,495,264,50.0
521,mdb321,D,NORM,NaN,604,271,50.0


In [232]:
## CANNNY
mean = []
var = []
std = []
max_v = []
min_v = []
min_coor_x = []
min_coor_y = []
max_coor_x = []
max_coor_y = []
for index, row in df_final.iterrows():
    path = 'canny/{name}.png'.format(name=row['ID'])
    im = cv2.imread('canny/{name}.png'.format(name=row['ID']))
    h,w,d = im.shape
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    #c = row['severity']
    mask = np.zeros((h,w), np.uint8)
    cv2.circle(mask,(x,1024-y),r ,1,thickness=-1)
    masked_data = cv2.bitwise_and(im, im, mask=mask)
    
    mean.append(cv2.meanStdDev(im, mask = mask)[0][0])
    var.append(np.power(cv2.meanStdDev(im, mask = mask)[1][0], 2))
    std.append(cv2.meanStdDev(im, mask = mask)[1][0])
    
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(gray, mask = mask)
    
    max_v.append(max_val)
    min_v.append(min_val)
    min_coor_x.append(min_loc[0])
    min_coor_y.append(min_loc[1])
    max_coor_x.append(max_loc[0])
    max_coor_y.append(max_loc[1])


In [255]:
## Erode
mean = []
var = []
std = []
max_v = []
min_v = []
min_coor_x = []
min_coor_y = []
max_coor_x = []
max_coor_y = []
for index, row in df_final.iterrows():
    path = 'erode/{name}.png'.format(name=row['ID'])
    im = cv2.imread('erode/{name}.png'.format(name=row['ID']))
    h,w,d = im.shape
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    #c = row['severity']
    mask = np.zeros((h,w), np.uint8)
    cv2.circle(mask,(x,1024-y),r ,1,thickness=-1)
    masked_data = cv2.bitwise_and(im, im, mask=mask)
    
    mean.append(cv2.meanStdDev(im, mask = mask)[0][0])
    var.append(np.power(cv2.meanStdDev(im, mask = mask)[1][0], 2))
    std.append(cv2.meanStdDev(im, mask = mask)[1][0])
    
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(gray, mask = mask)
    
    max_v.append(max_val)
    min_v.append(min_val)
    min_coor_x.append(min_loc[0])
    min_coor_y.append(min_loc[1])
    max_coor_x.append(max_loc[0])
    max_coor_y.append(max_loc[1])


In [256]:
df_final['mean_erode'] = np.array(mean)
df_final['var_erode'] = np.array(var)
df_final['std_erode'] = np.array(std)
df_final['min_pixel_erode'] = np.array(min_v)
df_final['max_pixel_erode'] = np.array(max_v)
df_final['min_coor_x_erode'] = np.array(min_coor_x)
df_final['min_coor_y_erode'] = np.array(min_coor_y)
df_final['max_coor_x_erode'] = np.array(max_coor_x)
df_final['max_coor_y_erode'] = np.array(max_coor_y)

In [233]:
df_final['mean_canny'] = np.array(mean)
df_final['var_canny'] = np.array(var)
df_final['std_canny'] = np.array(std)
df_final['min_pixel_canny'] = np.array(min_v)
df_final['max_pixel_canny'] = np.array(max_v)
df_final['min_coor_x_canny'] = np.array(min_coor_x)
df_final['min_coor_y_canny'] = np.array(min_coor_y)
df_final['max_coor_x_canny'] = np.array(max_coor_x)
df_final['max_coor_y_canny'] = np.array(max_coor_y)

In [216]:
mean = []
var = []
std = []
max_v = []
min_v = []
min_coor_x = []
min_coor_y = []
max_coor_x = []
max_coor_y = []
for index, row in df_final.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    im = cv2.imread('all-mias/{name}.pgm'.format(name=row['ID']))
    h,w,d = im.shape
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    #c = row['severity']
    mask = np.zeros((h,w), np.uint8)
    cv2.circle(mask,(x,1024-y),r ,1,thickness=-1)
    masked_data = cv2.bitwise_and(im, im, mask=mask)
    
    mean.append(cv2.meanStdDev(im, mask = mask)[0][0])
    var.append(np.power(cv2.meanStdDev(im, mask = mask)[1][0], 2))
    std.append(cv2.meanStdDev(im, mask = mask)[1][0])
    
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(gray, mask = mask)
    
    max_v.append(max_val)
    min_v.append(min_val)
    min_coor_x.append(min_loc[0])
    min_coor_y.append(min_loc[1])
    max_coor_x.append(max_loc[0])
    max_coor_y.append(max_loc[1])


In [217]:
df_final['mean'] = np.array(mean)
df_final['var'] = np.array(var)
df_final['std'] = np.array(std)
df_final['min_pixel'] = np.array(min_v)
df_final['max_pixel'] = np.array(max_v)
df_final['min_coor_x'] = np.array(min_coor_x)
df_final['min_coor_y'] = np.array(min_coor_y)
df_final['max_coor_x'] = np.array(max_coor_x)
df_final['max_coor_y'] = np.array(max_coor_y)

In [257]:
df_final

,ID,background_tissue,class,severity,x,y,rad,mean,var,std,...,std_canny,mean_erode,var_erode,std_erode,min_pixel_erode,max_pixel_erode,min_coor_x_erode,min_coor_y_erode,max_coor_x_erode,max_coor_y_erode
0,mdb001,G,CIRC,B,535,425,197.0,150.781051,3205.504934,56.617179,...,112.945588,145.328215,3258.043762,57.079276,5.0,217.0,371,708,584,598
1,mdb002,G,CIRC,B,522,280,69.0,197.870827,764.537398,27.650269,...,103.247451,193.119138,904.499352,30.074896,58.0,226.0,569,794,528,676
2,mdb005,F,CIRC,B,477,133,30.0,149.605105,409.222292,20.229243,...,117.540999,142.003899,500.080453,22.362479,76.0,174.0,464,918,475,876
3,mdb005,F,CIRC,B,500,168,26.0,162.426214,80.178549,8.954248,...,116.307381,156.487977,94.573288,9.724880,126.0,173.0,493,831,521,861
4,mdb010,F,CIRC,B,525,425,33.0,184.385157,127.091607,11.273491,...,119.127056,178.766207,134.473062,11.596252,154.0,200.0,558,599,520,599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,mdb318,D,NORM,N,442,68,50.0,14.033015,0.505857,0.711236,...,12.200599,13.095857,0.230200,0.479791,11.0,14.0,434,949,442,906
308,mdb319,D,NORM,N,590,281,50.0,145.912938,28.130725,5.303841,...,118.449283,141.277884,21.720868,4.660565,130.0,159.0,585,776,639,743
309,mdb320,D,NORM,N,495,264,50.0,195.578840,470.810515,21.698168,...,108.471641,190.905163,514.108278,22.673956,145.0,224.0,486,809,524,720
310,mdb321,D,NORM,N,604,271,50.0,177.764946,30.464061,5.519426,...,118.201415,173.387763,24.682272,4.968126,163.0,191.0,624,738,555,747


In [258]:
df_final.to_csv("finErode.csv")

In [259]:
all_coor = []
def get_all_coor():
    for index, row in df_final.iterrows():
        coordinates = []
        for x in range( int(row['x']) - int(row['rad']) ,  int(row['x']) + int(row['rad']) ) :
            for y in range( int(row['y']) - int(row['rad']), int(row['y']) + int(row['rad']) ):
                dx = x - int(row['x'])
                dy = y - int(row['y'])
                distanceSquared = dx * dx + dy * dy
                if (distanceSquared <= row['rad']**2):
                    coordinates.append((x,y))
        all_coor.append(coordinates)

In [260]:
get_all_coor()

In [261]:
bdips = []
#original is all-mias
def bdip():    
    for index, row in df_final.iterrows():
        path = 'erode/{name}.png'.format(name=row['ID'])
        im = cv2.imread('erode/{name}.png'.format(name=row['ID']))
        s = 0
        for x, y in all_coor[index]:
            s += im[row['max_coor_y'],row['max_coor_x']] - im[y, x]
        bdips.append( (s[0]/(row['rad']**2*3.14*im[row['max_coor_y'],row['max_coor_x']]))[0] )    

In [80]:
im = cv2.imread('all-mias/mdb001.pgm')

In [262]:
bdip()
bdips

<ipython-input-261-1b548d193135>:10: RuntimeWarning: divide by zero encountered in true_divide
  bdips.append( (s[0]/(row['rad']**2*3.14*im[row['max_coor_y'],row['max_coor_x']]))[0] )


[7.625498936395408e-06,
 6.33241334961923e-05,
 1.0286541911486364e-05,
 0.0006198834817417039,
 7.347835079428774e-06,
 2.575871112776321e-05,
 7.647594754973613e-05,
 3.73206478234761e-06,
 8.977526429837809e-05,
 3.299530435904793e-05,
 3.9992298549760605e-06,
 0.00020886054354180163,
 3.983378076649275e-05,
 1.048907945702727e-05,
 8.287007876037281e-05,
 4.874063936021087e-05,
 0.00019614146935280626,
 0.0002003039973707296,
 9.99108351302482e-05,
 0.00027080896052688593,
 0.0006746834281557642,
 0.001056656444129905,
 1.7397988402825692e-06,
 0.00015842181554134163,
 4.441652301470737e-05,
 0.00033023306311522874,
 3.2899731642694825e-05,
 2.7178114819417913e-05,
 6.2678582943939e-05,
 6.802544777580934e-05,
 2.8069747709107593e-05,
 7.533072023517883e-05,
 5.388545643137015e-06,
 2.3560427998460527e-05,
 0.00012307302668685216,
 2.4836178579197326e-06,
 1.0386054808723436e-06,
 2.56053344087921e-05,
 7.325930559656532e-06,
 8.797656426730095e-06,
 0.00021182418403032185,
 1.3646

In [263]:
df_final['bdips_erode'] = np.array(bdips)

In [264]:
df_final.to_csv("final_with_erode.csv")

In [265]:
df_final

,ID,background_tissue,class,severity,x,y,rad,mean,var,std,...,mean_erode,var_erode,std_erode,min_pixel_erode,max_pixel_erode,min_coor_x_erode,min_coor_y_erode,max_coor_x_erode,max_coor_y_erode,bdips_erode
0,mdb001,G,CIRC,B,535,425,197.0,150.781051,3205.504934,56.617179,...,145.328215,3258.043762,57.079276,5.0,217.0,371,708,584,598,0.000008
1,mdb002,G,CIRC,B,522,280,69.0,197.870827,764.537398,27.650269,...,193.119138,904.499352,30.074896,58.0,226.0,569,794,528,676,0.000063
2,mdb005,F,CIRC,B,477,133,30.0,149.605105,409.222292,20.229243,...,142.003899,500.080453,22.362479,76.0,174.0,464,918,475,876,0.000010
3,mdb005,F,CIRC,B,500,168,26.0,162.426214,80.178549,8.954248,...,156.487977,94.573288,9.724880,126.0,173.0,493,831,521,861,0.000620
4,mdb010,F,CIRC,B,525,425,33.0,184.385157,127.091607,11.273491,...,178.766207,134.473062,11.596252,154.0,200.0,558,599,520,599,0.000007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,mdb318,D,NORM,N,442,68,50.0,14.033015,0.505857,0.711236,...,13.095857,0.230200,0.479791,11.0,14.0,434,949,442,906,0.002019
308,mdb319,D,NORM,N,590,281,50.0,145.912938,28.130725,5.303841,...,141.277884,21.720868,4.660565,130.0,159.0,585,776,639,743,0.000043
309,mdb320,D,NORM,N,495,264,50.0,195.578840,470.810515,21.698168,...,190.905163,514.108278,22.673956,145.0,224.0,486,809,524,720,0.000036
310,mdb321,D,NORM,N,604,271,50.0,177.764946,30.464061,5.519426,...,173.387763,24.682272,4.968126,163.0,191.0,624,738,555,747,0.000087


In [246]:
df_final

,ID,background_tissue,class,severity,x,y,rad,mean,var,std,min_pixel,max_pixel,min_coor_x,min_coor_y,max_coor_x,max_coor_y,bdips,mean_canny,var_canny,std_canny
0,mdb001,G,CIRC,B,535,425,197.0,150.781051,3205.504934,56.617179,5.0,221.0,373,710,487,632,0.000007,68.343054,12756.705742,112.945588
1,mdb002,G,CIRC,B,522,280,69.0,197.870827,764.537398,27.650269,75.0,229.0,571,792,531,679,0.000055,52.692153,10660.036075,103.247451
2,mdb005,F,CIRC,B,477,133,30.0,149.605105,409.222292,20.229243,84.0,187.0,481,920,476,876,0.000087,78.099965,13815.886498,117.540999
3,mdb005,F,CIRC,B,500,168,26.0,162.426214,80.178549,8.954248,128.0,181.0,520,841,522,859,0.000578,75.261669,13527.406777,116.307381
4,mdb010,F,CIRC,B,525,425,33.0,184.385157,127.091607,11.273491,155.0,208.0,557,599,519,598,0.000287,82.057788,14191.255387,119.127056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,mdb318,D,NORM,N,442,68,50.0,14.033015,0.505857,0.711236,11.0,35.0,436,951,467,965,0.000641,0.585086,148.854615,12.200599
308,mdb319,D,NORM,N,590,281,50.0,145.912938,28.130725,5.303841,130.0,168.0,587,778,637,745,0.000114,80.319312,14030.232648,118.449283
309,mdb320,D,NORM,N,495,264,50.0,195.578840,470.810515,21.698168,146.0,227.0,451,745,527,722,0.000004,60.491396,11766.096962,108.471641
310,mdb321,D,NORM,N,604,271,50.0,177.764946,30.464061,5.519426,163.0,199.0,626,740,555,746,0.000026,79.701721,13971.574509,118.201415


In [106]:
df_final.to_csv("final.csv")

In [185]:
df_final.to_csv("final2.csv")